In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
# using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo
using Distributions

  Activating project at `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
# import piano model
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
# model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")
data = init_data(model)

MuJoCo Data object

Model info

In [4]:
println("number of geoms: ",model.ngeom)
println("number of controls: ",model.nu)
println("number of states: ",model.nv+model.nq)
forward!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [42,50,58,68,77]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        42
        43
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        50
        51
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        58
        59
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        68
        69
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        77
        78
""";

number of geoms: 79
number of controls: 7
number of states: 46
Simulation timestep: 0.01
Positions of joints: [0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
Cartesion position of joints: [0.0 0.0 0.0; -0.127 0.0 0.02; 0.0 -0.094 0.01125; -0.03 -0.08224999999999999 0.026000000000000002; 0.0 -0.07050000000000001 0.01125; 0.0 -0.047 0.01125; -0.03 -0.03474999999999999 0.026000000000000002; 0.0 -0.023499999999999993 0.01125; -0.03 -0.011249999999999996 0.026000000000000002; 0.0 0.0 0.01125; 0.0 0.023499999999999993 0.01125; -0.03 0.035750000000000004 0.026000000000000002; 0.0 0.047000000000000014 0.01125; -0.03 0.05925 0.026000000000000002; 0.0 0.07050000000000001 0.01125; -0.03 0.08275 0.026000000000000002; 0.0 0.094 0.01125; 0.3938251578954367 0.14997497683550026 0.13062742818321735; 0.29337550270087137 0.15000084946805123 0.12910710828129915; 0.3072543776776233 0.15000002300031276 0.1299999783510228; 0.21900000000000

Visualize target finger position

In [5]:
# control indices:
# 1: wrist left right
# 2: wrist up down
# 8, 9, 10: index finger joints
# 21: whole hand left right 
# 22: whole hand front back

In [6]:
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states = zeros(nx, 19)
states_geom = zeros(3, 19)
U = zeros(model.nu)
U[4] = 0.664
U[5] = 1.1
U[2] = 0.21
# U[1] = -0.234
record_t_3 = zeros(nx)
data = init_data(model)
step!(model, data)
println(get_physics_state(model, data))
data = init_data(model)
step!(model, data)
println(get_physics_state(model, data))

step!(model, data)
println("max",maximum(get_physics_state(model, data)))
println("min",minimum(get_physics_state(model, data)))

states[:,1] = get_physics_state(model, data)
states_geom[:,1] = data.geom_xpos[finger_geom_indices[1],:]
for t in 1:18
    # println(t,data.geom_xpos[finger_geom_indices[1],:])
    data.ctrl .= U
    step!(model, data)
    
    states[:,t+1] = get_physics_state(model, data)
    states_geom[:,t+1] = data.geom_xpos[finger_geom_indices[1],:]
end
# println(data.geom_xpos[finger_geom_indices[1],:])
reset!(model, data)

data.ctrl .= U
data.qpos .= states[:,1][1:model.nq]
data.qvel .= states[:,1][model.nq+1:end]
forward!(model, data)
# println("hi")
println("hi",data.geom_xpos[finger_geom_indices[1],:])
for t in 1:19
    println(t,states_geom[:,t])
end
init_visualiser()
visualise!(model, data, trajectories = [states])

[-0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.008912216587082428, -4.2341315644610154e-5, -0.042233680988673, 0.004593844064341336, 0.0034952271077786437, 0.0017690537583294683, 0.00045559012633187955, 0.00010943869295629263, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0

## Test DIRCOL: moving index finger left right

In [17]:
global mj_access_count = 0

0

In [7]:
function normalize_states(model, x)
    # get the states and normalize according to joint and control ranges
    for j in 
        x_l[idx.x[i][j]] = model.jnt_range[j,1]
        x_u[idx.x[i][j]] = model.jnt_range[j,2]
end

normalize_states (generic function with 1 method)

In [27]:
function wrapped_mj_step(model, data, x, uk, forward)
    # given the current model and data. set the state and control to the model and perform a forward step
    # data.ctrl .= zeros(model.nu)
    global mj_access_count += 1
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        # data.qpos .= x[1:model.nq]
        # data.qvel .= x[(model.nq + 1):end]
        data.qpos .= x[1:model.nq]
        data.qvel .= x[model.nq+1:end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= ForwardDiff.value.(uk)

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end

    # return updated state k + 1
    # xkp1 = zeros(model.nq + model.nv) 
    # xkp1 .= get_physics_state(model, data)
    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return data.geom_xpos[finger_geom_indices[1],:], get_physics_state(model, data)
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg, xic = params.idx, params.N, params.xg, params.xic
    model = params.model
    data = init_data(model)
    step!(model, data)
    Q, R, R_d, Q_reg, Q_coord = params.Q, params.R, params.R_d, params.Q_reg, params.Q_coord
    coord_interpolation = params.coord_interpolation

    # stage cost
    cost = 0.0
    xi = Z[idx.x[1]]
    ui = Z[idx.u[1]]

    finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
    cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui
    cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
    cost += 0.5*(finger_geom_xpos-coord_interpolation[1])'*Q_coord*(finger_geom_xpos-coord_interpolation[1])
    # cost += 0.5*(xi)'*Q_reg*(xi)
    for i = 2:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        ui_1 = Z[idx.u[i-1]]
        finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
        # @show xi-xg
        # actual cost
        cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui#+0.5*(ui-ui_1)'*R_d*(ui-ui_1)
        # regularizer
        cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
        cost += 0.5*(finger_geom_xpos-coord_interpolation[i])'*Q_coord*(finger_geom_xpos-coord_interpolation[i])
    end
    cost += 0.5*(finger_geom_xpos-coord_interpolation[N])'*Q_coord*(finger_geom_xpos-coord_interpolation[N])
    # # terminal cost 
    # xf = Z[idx.x[N]]
    # # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    # finger_geom_xpos, _ = wrapped_mj_step(model, data, xf, 0, false)
    # cost += 0.5*(finger_geom_xpos-xg)'*Qf*(finger_geom_xpos-xg)

    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    step!(model, data)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        _,  xip1_mujoco = wrapped_mj_step(model, data, xi, ui, false)

        c[idx.c[i]] = (xip1_mujoco - xip1)
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    
    # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    con_1 =  Z[idx.x[1]] - xic
    
    con_2 =  terminal_pose_index - xg
    # con_2[3] = con_2[3] - 0.5

    return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    # return zeros(eltype(Z), 0)
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    
    # # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    # terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    # con_1 =  Z[idx.x[1]] - xic
    
    # cart_z_end =  terminal_pose_index 

    # return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    return zeros(eltype(Z), 0)
    # return cart_z_end
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [28]:
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
for j = 1:model.nu
    println(j," upper:",model.actuator_ctrlrange[j,1])
    println(j," lower:",model.actuator_ctrlrange[j,2])
end

1 upper:-0.523599
1 lower:0.174533
2 upper:-0.698132
2 lower:0.488692
3 upper:-0.349066
3 lower:0.349066
4 upper:-0.261799
4 lower:1.5708
5 upper:0.0
5 lower:3.1415
6 upper:-0.25525
6 lower:-0.04475
7 upper:0.0
7 lower:0.06


In [37]:
global mj_access_count = 0
function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand_small/scene.xml")
    # model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")

    data = init_data(model)

    # reset the model and data
    reset!(model, data)

    # initiate time and time steps
    dt = 0.01
    tf = 0.1
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))

    # Q = 100*diagm(ones(3))
    Q = 1*diagm([5,5,30])
    Q_coord = 1*diagm(ones(3))
    Q_reg = 1*diagm(ones(model.nq + model.nv))
    R = 1*diagm(ones(model.nu))
    # udifference regularizer
    R_d = 10*diagm(ones(model.nu))

    idx = create_idx(model.nq + model.nv, model.nu, N)
    # initial and goal states
    step!(model, data)
    xic = get_physics_state(model, data)
    xic_geom = data.geom_xpos[finger_geom_indices[1],:]
    U_ref = zeros(model.nu)
    U_ref[4] = 0.664
    U_ref[5] = 1.1
    U_ref[2] = 0.21
    for t in 1:N-1
        data.ctrl .= U_ref
        step!(model, data)
    end
    # initial and goal state
    xg_physics = get_physics_state(model, data)
    xg = data.geom_xpos[finger_geom_indices[1],:]
    # reset the model and data

    
    # linear-wise interpolation of xi and xg
    coord_interpolation = [zeros(3) for _ in 1:N]
    for i in 1:N
        coord_interpolation[i] = xic_geom + (i-1)/(N-1)*(xg-xic_geom)
    end

    reset!(model, data)
    params = (Q = Q, R = R, R_d = R_d, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, Q_reg=Q_reg, coord_interpolation=coord_interpolation, Q_coord=Q_coord)

    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        for j = 1:model.nq
            # hands
            x_l[idx.x[i][j]] = model.jnt_range[j,1]
            x_u[idx.x[i][j]] = model.jnt_range[j,2]
        end
    end
    for i = 1:(N-1)
        for j = 1:model.nu
            x_l[idx.u[i][j]] = model.actuator_ctrlrange[j,1]
            x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        end
        # for j in [2,5,4]
        #     x_l[idx.u[i][j]] = 0
        #     x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        # end
    end
    

    # inequality constraints
    # c_l = xg .- 0.005
    # c_u = xg .+ 0.005
    c_l = zeros(0)
    c_u = zeros(0)
    # c_l = -0.01*ones(idx.nx+3+idx.nx*(N-1))
    # c_u = 0.01*ones(idx.nx+3+idx.nx*(N-1))
    # initial guess 
    z0 = randn(idx.nz)
    z0[idx.x[1]] .= xic
    for i = 2:N
        # z0[idx.x[i]] .= xic + ((xg_physics-xic)/(N))*(i)
        z0[idx.x[i]] .= xic
        # z0[idx.x[i]] .= xg_physics
        # for j = 1:model.nq
        #     z0[idx.x[i][j]] = rand(Uniform(model.jnt_range[j,1],model.jnt_range[j,2]))
        # end
    end
    println(length(robohand_inequality_constraints(params,z0)))
    println(length(c_u))
    for i = 1:N-1
        for j = 1:model.nu
            z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        end
        # for j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
    end
    # diff type
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-4, c_tol = 1e-4, max_iters = 500, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [38]:
X, U, t_vec, params= solve_finger_moving(verbose=true)

0
0
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   293184
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      576
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      323
                     variables with only upper bounds:        0
Total number of equality constraints.................:      509
Total number of inequality constraints...

([[-9.999999973841396e-9, -9.999999995216636e-9, -9.999999973841394e-9, -9.999999973841396e-9, -9.99999999521664e-9, -9.999999973841404e-9, -9.999999995216628e-9, -9.999999973841406e-9, -9.999999973841394e-9, -9.99999999521663e-9  …  -0.16775113564067246, -0.030396975279481788, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0.045559012633187956, 0.010943869295629263], [-9.999999967326378e-9, -9.999999994291767e-9, -9.99999996732638e-9, -9.999999967326378e-9, -9.999999994291767e-9, -9.999999967326419e-9, -9.999999994291767e-9, -9.999999967326422e-9, -9.999999967326378e-9, -9.999999994291769e-9  …  -0.07132391583031082, -0.01178750697479205, -0.8230396000771748, 0.01404877007042548, -4.100244409197771, 0.8558775050866861, 0.13608729085219, 9.471926176571259, 12.817514625286176, 13.745398219858316], [-9.999999964802686e-9, -9.999999993908348e-9, -9.999999964802686e-9, -9.999999964802686e-9, -9.999999993908346e-9

In [39]:
mj_access_count 

2579654

In [40]:
X

11-element Vector{Vector{Float64}}:
 [-9.999999973841396e-9, -9.999999995216636e-9, -9.999999973841394e-9, -9.999999973841396e-9, -9.99999999521664e-9, -9.999999973841404e-9, -9.999999995216628e-9, -9.999999973841406e-9, -9.999999973841394e-9, -9.99999999521663e-9  …  -0.16775113564067246, -0.030396975279481788, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0.045559012633187956, 0.010943869295629263]
 [-9.999999967326378e-9, -9.999999994291767e-9, -9.99999996732638e-9, -9.999999967326378e-9, -9.999999994291767e-9, -9.999999967326419e-9, -9.999999994291767e-9, -9.999999967326422e-9, -9.999999967326378e-9, -9.999999994291769e-9  …  -0.07132391583031082, -0.01178750697479205, -0.8230396000771748, 0.01404877007042548, -4.100244409197771, 0.8558775050866861, 0.13608729085219, 9.471926176571259, 12.817514625286176, 13.745398219858316]
 [-9.999999964802686e-9, -9.999999993908348e-9, -9.999999964802686e-9, -9.999999

In [41]:
U
# U2[4] = 0.664
# U2[5] = 1.1
# U2[2] = 0.21

10-element Vector{Vector{Float64}}:
 [-0.28721874139087583, 0.09891027501733299, 0.0067361194171547175, 0.9178539326656333, 2.000587658818366, -0.14712423167507518, 0.021161234983813813]
 [-0.2994724823228026, 0.12786591797821134, 0.015991382770701865, 0.7801756504988435, 1.4902609690735433, -0.13681459055712328, 0.023296683379562463]
 [-0.29328557766350427, 0.19057650942723223, 0.07395731318547984, 0.7544379552755254, 1.3752022234906907, -0.13332741429846462, 0.024850694005092575]
 [-0.3061221232759382, 0.2046069104901218, 0.07646302245463407, 0.8247981003441925, 1.3885971804006714, -0.12953872138069566, 0.026864397856099655]
 [-0.3006141385475506, 0.11344549326300706, 0.12852539992506715, 1.036893619647609, 1.5128785272334069, -0.12733497590778406, 0.03038599583693755]
 [-0.36140655014863154, -0.00464698612397942, 0.02665854938221919, 0.5671030219310692, 1.0042037546616427, -0.12953864584950164, 0.025543884863269053]
 [-0.3285569323911857, -0.0040141893771961715, -0.03336217472000574

In [42]:
# instantiate model and data
# model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

# reset the model and data j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
dt = 0.01
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
reset!(model, data)
step!(model, data)
states[:,1] = get_physics_state(model, data)
for t in 1:N-1
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println("end using controls:",data.geom_xpos[finger_geom_indices[1],:])
states3 = zeros(nx, N)
for t in 1:N
    states3[:,t] = X[t][:]
end

reset!(model, data)
data.ctrl .=0
data.qpos .= states3[1:model.nq,N]
data.qvel .= states3[model.nq+1:end,N]
forward!(model, data)
final_cart_fmincon = data.geom_xpos[finger_geom_indices[1],:]
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states2 = zeros(nx, N)
states2[:,1] = get_physics_state(model, data)
states2_coord = zeros(3,N)
states2_coord[:,1] = data.geom_xpos[finger_geom_indices[1],:]
U2 = zeros(model.nu)
U2[4] = 0.664
U2[5] = 1.1
U2[2] = 0.21
for t in 1:N-1
    data.ctrl .= U2
    step!(model, data)
    states2[:,t+1] = get_physics_state(model, data)
    states2_coord[:,t+1] = data.geom_xpos[finger_geom_indices[1],:]
end
println(data.geom_xpos[finger_geom_indices[1],:])
println(norm((final_cart_fmincon[3]-data.geom_xpos[finger_geom_indices[1],3]),2))
for t in 1:N
    println(states2_coord[:,t])
end

end using controls:[0.017801679975391484, 0.10115972195340027, 0.06325340955658955]
[0.025395381454867045, 0.07193963679211916, 0.056461397004888794]
0.005663976712741492
[0.0, 0.0, 0.0]
[0.0004900000000000147, 0.11699999999999999, 0.12000000000000001]
[-0.00039745522853294746, 0.11618415701496942, 0.11305629845488538]
[0.0004147145284350614, 0.11277704589079526, 0.102973328879172]
[0.0030054994381000857, 0.10707991862841891, 0.09221868253682486]
[0.006791496182904927, 0.09991124975829715, 0.08221422542634418]
[0.011162703930066919, 0.0921847080746731, 0.07363110262822647]
[0.015600942053661448, 0.08457642963920377, 0.06664445126841494]
[0.020412581984727694, 0.07794182133290084, 0.060768332182999533]
[0.023402564163286798, 0.07417287845560569, 0.058091885765530954]
[0.025395381454867045, 0.07193963679211916, 0.056461397004888794]


In [43]:
init_visualiser()
visualise!(model, data, trajectories = [states, states2])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [45]:
init_visualiser()
visualise!(model, data, trajectories = [states2])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [46]:
init_visualiser()
visualise!(model, data, trajectories = [states3])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
